# Итоговый проект. Улицы. Объекты культурного наследия.

In [ ]:
# DONE todo собрать установку всех необходимых модулей в одном месте
!pip install folium -U
!pip install geopandas mapclassify
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# DONE todo собрать импорты всех модулей в одном месте
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [53]:
# DONE todo указать через константу, чтобы везде была одна и та же подложка карт. Сейчас в работе название захардкожено 
TILES = "CartoDB positron"

# DONE todo собрать все константы в одном месте
# DONE todo указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Василеостровский район, Санкт-Петербург'  # например, 'Адмиралтейский район, Санкт-Петербург'
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
# DONE todo указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/pashkalini/PythonUrbanFinalTask/main/kgiop_objects.geojson"
STREET_BUFFER = 100

In [ ]:
def get_google_drive_download_url(url: str) -> str:
    """
    Функция возвращает ссылку для скачивания с google drive.
     Например: "https://drive.google.com/file/d/1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp/view?usp=sharing" ->
     "https://drive.google.com/uc?export=download&id=1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp"
    """
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

## Территория

### Загрузка территории из OSM (Extract)

In [ ]:
territory = ox.geocode_to_gdf(TERRITORY_NAME)  # подгружаем с OSM и получаем GeoDataFrame
territory.explore(tiles=TILES)

### Обработка данных с территорией (Transform)

In [ ]:
territory

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((30.17787 59.94437, 30.17790 59.94403...",59.964376,59.91726,30.316861,30.177873,298216825,relation,1114252,59.939275,30.243876,"Василеостровский район, Saint Petersburg, Nort...",boundary,administrative,0.600226


Добавим новый столбец с "Название территории"

In [34]:
# DONE todo добавить столбец Название территории и удалить все остальные (кроме столбца geometry)
territory['Название территории'] = TERRITORY_NAME

Оставим только столбец *geometry* и новый с названием территории

In [96]:
#territory = territory[['Название территории', 'geometry']]
territory = territory.loc[:, ('Название территории', 'geometry')]
territory

,Название территории,geometry
0,"Василеостровский район, Санкт-Петербург","POLYGON ((30.17787 59.94437, 30.17790 59.94403..."


## Улицы

### Загрузка файла с улицами из google disk (Extract)

 Отфильтруем улицы по маске геометрии территории

In [27]:
url = get_google_drive_download_url(STREETS_FILE_URL)
# DONE todo отфильтровать улицы по маске геометрии территории полученной ранее
gdf_streets = gpd.read_file(url, mask=territory.geometry)
gdf_streets

,type,id,tags,geometry
0,way,4454390,"{'foot': 'no', 'highway': 'unclassified', 'lan...","LINESTRING (3366312.890 8383418.064, 3366318.9..."
1,way,4454543,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3364065.060 8389870.685, 3364042.8..."
2,way,4454721,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3366216.843 8386214.551, 3366217.3..."
3,way,4454767,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3363246.227 8390001.823, 3363259.4..."
4,way,4455074,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3364793.702 8386752.080, 3364743.2..."
...,...,...,...,...
5494,way,983003449,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3365411.091 8388516.042, 3365410.5..."
5495,way,983003450,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3365780.426 8389603.970, 3365742.5..."
5496,way,983133694,{'highway': 'service'},"LINESTRING (3368553.384 8387054.742, 3368548.4..."
5497,way,983133697,"{'highway': 'service', 'layer': '-1', 'tunnel'...","LINESTRING (3368611.225 8387181.673, 3368580.9..."


### Обработка данных с улицами (Transform)

Посмотрим все данные по улицам района

In [28]:
tags = pd.json_normalize(gdf_streets["tags"])
tags

,foot,highway,lanes,lit,maxspeed,name,name:en,name:ru,surface,trolley_wire,...,cycleway:right,maxweight,surface:note,fixme,proposed,motor_vehicle,turn:backward,turn:forward,lane_markings,parking:condition:right
0,no,unclassified,2,yes,RU:urban,Кожевенная линия,Kozhevennaya Line,Кожевенная линия,asphalt,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no,secondary,3,yes,RU:urban,улица Нахимова,Nahimova Street,улица Нахимова,asphalt,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5494,no,residential,1,yes,RU:urban,NaN,NaN,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5495,no,residential,1,yes,RU:urban,NaN,NaN,NaN,asphalt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5496,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5497,NaN,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5499 entries, 0 to 5498
Data columns (total 81 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   foot                         791 non-null    object
 1   highway                      5499 non-null   object
 2   lanes                        900 non-null    object
 3   lit                          869 non-null    object
 4   maxspeed                     887 non-null    object
 5   name                         826 non-null    object
 6   name:en                      642 non-null    object
 7   name:ru                      642 non-null    object
 8   surface                      2103 non-null   object
 9   trolley_wire                 206 non-null    object
 10  living_street                1536 non-null   object
 11  oneway                       761 non-null    object
 12  lanes:forward                10 non-null     object
 13  postal_code                  35 n

In [30]:
extract_tags = [
    'name'  # DONE todo указать в списке строку "name"
]

gdf_streets = gdf_streets.join(tags[extract_tags])
gdf_streets

,type,id,tags,geometry,name
0,way,4454390,"{'foot': 'no', 'highway': 'unclassified', 'lan...","LINESTRING (3366312.890 8383418.064, 3366318.9...",Кожевенная линия
1,way,4454543,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3364065.060 8389870.685, 3364042.8...",NaN
2,way,4454721,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3366216.843 8386214.551, 3366217.3...",NaN
3,way,4454767,"{'highway': 'service', 'living_street': 'yes'}","LINESTRING (3363246.227 8390001.823, 3363259.4...",NaN
4,way,4455074,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3364793.702 8386752.080, 3364743.2...",улица Нахимова
...,...,...,...,...,...
5494,way,983003449,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3365411.091 8388516.042, 3365410.5...",NaN
5495,way,983003450,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3365780.426 8389603.970, 3365742.5...",NaN
5496,way,983133694,{'highway': 'service'},"LINESTRING (3368553.384 8387054.742, 3368548.4...",NaN
5497,way,983133697,"{'highway': 'service', 'layer': '-1', 'tunnel'...","LINESTRING (3368611.225 8387181.673, 3368580.9...",NaN


Удалим строки с отстутсвующими названиями улиц

In [36]:
gdf_streets.dropna(
    how="all",  # удалить если отсутствуют значения во всех указанных столбцах
    subset=["name"],  # столбцы, в которых надо искать пропуски
    inplace=True
)
gdf_streets

,type,id,tags,geometry,name
0,way,4454390,"{'foot': 'no', 'highway': 'unclassified', 'lan...","LINESTRING (3366312.890 8383418.064, 3366318.9...",Кожевенная линия
4,way,4455074,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3364793.702 8386752.080, 3364743.2...",улица Нахимова
6,way,4455758,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3368203.462 8387924.925, 3368216.1...",16-17-я линии В.О.
7,way,4456100,"{'highway': 'residential', 'lanes': '1', 'lit'...","LINESTRING (3368114.551 8385036.156, 3368124.6...",Клубный переулок
10,way,4456704,"{'foot': 'no', 'highway': 'residential', 'lane...","LINESTRING (3367355.174 8388720.917, 3367360.5...",переулок Декабристов
...,...,...,...,...,...
5412,way,946864884,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3362469.039 8385708.171, 3362340.7...",бульвар Александра Грина
5415,way,951667505,"{'foot': 'no', 'highway': 'tertiary', 'lanes':...","LINESTRING (3369235.249 8388117.669, 3369199.1...",набережная реки Смоленки
5416,way,951667506,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3369481.065 8388327.601, 3369496.5...",9-я линия В.О.
5456,way,957802292,"{'bicycle': 'designated', 'check_date': '2017-...","LINESTRING (3368901.691 8384926.614, 3368882.4...",Большой проспект В.О.


Проверим наличие дубликатов

In [37]:
# DONE todo проверить наличие дубликатов по полям "id", "geometry", "name"
gdf_streets.duplicated(subset=["id", "geometry", "name"]).value_counts()

False    826
dtype: int64

Посмотрим, являются ли названия улиц уникальными

In [38]:
gdf_streets["name"].is_unique

False

При помощи метода *dissolve* группируем геометрии по столбцу *name* и объединяем геометрии с одинаковыми названиями

In [39]:
gdf_streets = gdf_streets.dissolve(by="name")  # DONE todo сгруппировать и объединить геометрии с одинаковыми названиями
gdf_streets

,geometry,type,id,tags
name,,,,
1-я линия В.О.,"MULTILINESTRING ((3371228.480 8388213.098, 337...",way,128224708,"{'foot': 'no', 'highway': 'primary', 'lanes': ..."
10-11-я линии В.О.,"MULTILINESTRING ((3369235.249 8388117.669, 336...",way,31364918,"{'foot': 'no', 'highway': 'residential', 'lane..."
12-13-я линии В.О.,"MULTILINESTRING ((3370596.664 8385143.832, 337...",way,240799870,"{'foot': 'no', 'highway': 'residential', 'lane..."
14-15-я линии В.О.,"LINESTRING (3369473.640 8386432.496, 3369476.8...",way,173075244,"{'foot': 'no', 'highway': 'residential', 'lane..."
14-я линия В.О.,"LINESTRING (3369473.640 8386432.496, 3369470.2...",way,31389152,"{'foot': 'no', 'highway': 'tertiary', 'lanes':..."
...,...,...,...,...
улица Нахимова,"MULTILINESTRING ((3364793.702 8386752.080, 336...",way,4455074,"{'foot': 'no', 'highway': 'secondary', 'lanes'..."
улица Одоевского,"MULTILINESTRING ((3368096.740 8388961.412, 336...",way,4458561,"{'foot': 'no', 'highway': 'residential', 'lane..."
улица Репина,"MULTILINESTRING ((3371298.233 8387498.912, 337...",way,25091782,"{'foot': 'yes', 'highway': 'residential', 'lan..."


Меняем название первого столбца (индекса)  с *name* на *Название улицы*

In [41]:
gdf_streets.index.rename("Название улицы", inplace=True)
gdf_streets.head()

,geometry,type,id,tags
Название улицы,,,,
1-я линия В.О.,"MULTILINESTRING ((3371228.480 8388213.098, 337...",way,128224708,"{'foot': 'no', 'highway': 'primary', 'lanes': ..."
10-11-я линии В.О.,"MULTILINESTRING ((3369235.249 8388117.669, 336...",way,31364918,"{'foot': 'no', 'highway': 'residential', 'lane..."
12-13-я линии В.О.,"MULTILINESTRING ((3370596.664 8385143.832, 337...",way,240799870,"{'foot': 'no', 'highway': 'residential', 'lane..."
14-15-я линии В.О.,"LINESTRING (3369473.640 8386432.496, 3369476.8...",way,173075244,"{'foot': 'no', 'highway': 'residential', 'lane..."
14-я линия В.О.,"LINESTRING (3369473.640 8386432.496, 3369470.2...",way,31389152,"{'foot': 'no', 'highway': 'tertiary', 'lanes':..."


Удалим столбцы *id*, *type* и *tags*

In [42]:
# todo удалить столбцы "id", "type" и "tags"
drop_columns = [
    'id',
    'type',
    'tags'
]
gdf_streets.drop(columns=drop_columns, inplace=True)
gdf_streets.head()

,geometry
Название улицы,
1-я линия В.О.,"MULTILINESTRING ((3371228.480 8388213.098, 337..."
10-11-я линии В.О.,"MULTILINESTRING ((3369235.249 8388117.669, 336..."
12-13-я линии В.О.,"MULTILINESTRING ((3370596.664 8385143.832, 337..."
14-15-я линии В.О.,"LINESTRING (3369473.640 8386432.496, 3369476.8..."
14-я линия В.О.,"LINESTRING (3369473.640 8386432.496, 3369470.2..."


Отобразим полученную геометрию (улицы района)

In [47]:
gdf_streets.explore(tiles=TILES)

## Объекты культурного наследия

### Загрузка объектов культурного наследия

In [101]:
gdf_kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory)
gdf_kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1260,—,Дом А.Р. Гешвенда,1880-1881,"9-я линия В.О., 32",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370436.885 8386572.161)
1,1021,Дом И.В. Голубина (И.И. Зайцевского),Флигель,нач. 19 в.; 1834,"Репина ул., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3371907.580 8386491.569)
2,1015,—,Дом Г.И. Головкина (А.Н. Чичагова),1720-е - 1730-е; 1878,"1-я линия В.О., 10; Репина ул., 9",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372020.006 8386392.023)
3,1016,—,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1722-1741, 1807-1809","1-я линия В.О., 12",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3372009.644 8386417.444)
4,1017,—,Дом А. Сверчкова (Кушелевой),1730-1740-е; 1815; 1863,"1-я линия В.О., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 139-р от 23.04.2021,Памятник,POINT (3372026.660 8386473.431)
...,...,...,...,...,...,...,...,...,...
777,9441,Комплекс зданий Французского благотворительног...,Ограда,1905,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Памятник,POINT (3369220.867 8386925.034)
778,9442,Комплекс зданий Французского благотворительног...,Сад,кон. 19 - нач. 20 в.,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Ансамбль,POINT (3369290.775 8386927.924)
779,9517,—,Главное здание Государственного оптического ин...,1947-1956,"Кадетская линия, 5, корп. 2, лит. В",объект культурного наследия регионального знач...,Распоряжение КГИОП от 01.07.2019 № 365-р,Памятник,POINT (3372240.630 8387172.627)
780,9605,Два жилых флигеля,Жилой флигель,кон. 18 в.; 1848,"4-я линия В.О., 39, лит. Б",объект культурного наследия регионального знач...,Распоряжение КГИОП от 05.08.2019 № 437-р,Памятник,POINT (3370714.885 8387675.618)


Оставим только столбцы object_name, object_location, object_type и geometry

In [102]:
# DONE todo оставить только столбцы object_name, object_location, object_type
#gdf_kgiop_objects = gdf_kgiop_objects[['object_name', 'object_location', 'object_type']]
gdf_kgiop_objects = gdf_kgiop_objects.loc[:, ('object_name', 'object_location', 'object_type', 'geometry')]
gdf_kgiop_objects

,object_name,object_location,object_type,geometry
0,Дом А.Р. Гешвенда,"9-я линия В.О., 32",Памятник,POINT (3370436.885 8386572.161)
1,Флигель,"Репина ул., 17",Памятник,POINT (3371907.580 8386491.569)
2,Дом Г.И. Головкина (А.Н. Чичагова),"1-я линия В.О., 10; Репина ул., 9",Памятник,POINT (3372020.006 8386392.023)
3,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1-я линия В.О., 12",Памятник,POINT (3372009.644 8386417.444)
4,Дом А. Сверчкова (Кушелевой),"1-я линия В.О., 14, лит. А",Памятник,POINT (3372026.660 8386473.431)
...,...,...,...,...
777,Ограда,"14-я линия В.О., 57-61",Памятник,POINT (3369220.867 8386925.034)
778,Сад,"14-я линия В.О., 57-61",Ансамбль,POINT (3369290.775 8386927.924)
779,Главное здание Государственного оптического ин...,"Кадетская линия, 5, корп. 2, лит. В",Памятник,POINT (3372240.630 8387172.627)
780,Жилой флигель,"4-я линия В.О., 39, лит. Б",Памятник,POINT (3370714.885 8387675.618)


Переименуем столбцы *object_name*, *object_location*, *object_type* в русские названия

In [103]:
# DONE todo переименовать столбцы object_name, object_location, object_type в русские названия
rename_columns = {
    "object_name": "Название объекта",
    "object_location": "Местоположение объекта",
    "object_type": "Тип объекта"
}
gdf_kgiop_objects.rename(columns=rename_columns, inplace=True)
gdf_kgiop_objects

,Название объекта,Местоположение объекта,Тип объекта,geometry
0,Дом А.Р. Гешвенда,"9-я линия В.О., 32",Памятник,POINT (3370436.885 8386572.161)
1,Флигель,"Репина ул., 17",Памятник,POINT (3371907.580 8386491.569)
2,Дом Г.И. Головкина (А.Н. Чичагова),"1-я линия В.О., 10; Репина ул., 9",Памятник,POINT (3372020.006 8386392.023)
3,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1-я линия В.О., 12",Памятник,POINT (3372009.644 8386417.444)
4,Дом А. Сверчкова (Кушелевой),"1-я линия В.О., 14, лит. А",Памятник,POINT (3372026.660 8386473.431)
...,...,...,...,...
777,Ограда,"14-я линия В.О., 57-61",Памятник,POINT (3369220.867 8386925.034)
778,Сад,"14-я линия В.О., 57-61",Ансамбль,POINT (3369290.775 8386927.924)
779,Главное здание Государственного оптического ин...,"Кадетская линия, 5, корп. 2, лит. В",Памятник,POINT (3372240.630 8387172.627)
780,Жилой флигель,"4-я линия В.О., 39, лит. Б",Памятник,POINT (3370714.885 8387675.618)


Отобразим на карте объекты культурного наследия Василеостровского района

In [104]:
gdf_kgiop_objects.explore(tiles=TILES)

Используем ранее заданный буфер в 100 метров для подсчета кол-ва объектов культурного наследия, попадающих в буфер улицы


In [106]:
def get_contains_kgiop_objects(street) -> int:
    """ Функция для подсчета количества объектов культурного наследия попадающих в буффер улицы"""
    return sum(gdf_kgiop_objects["geometry"].within(street.buffer(STREET_BUFFER)))


gdf_streets["contains_kgiop_objects"] = gdf_streets["geometry"].apply(get_contains_kgiop_objects)
gdf_streets.nlargest(5, "contains_kgiop_objects")

,geometry,contains_kgiop_objects
Название улицы,,
4-5-я линии В.О.,"MULTILINESTRING ((3370879.560 8387254.041, 337...",49
1-я линия В.О.,"MULTILINESTRING ((3371228.480 8388213.098, 337...",42
8-я линия В.О.,"MULTILINESTRING ((3371178.720 8385438.273, 337...",38
9-я линия В.О.,"MULTILINESTRING ((3370822.954 8386044.168, 337...",37
Средний проспект В.О.,"MULTILINESTRING ((3366331.992 8384554.141, 336...",37


Визуализируем полученные выше данные

In [107]:
gdf_streets.explore("contains_kgiop_objects", tiles=TILES)

Посчитаем плотность расположения объектов культурного наследия на улице

In [108]:
gdf_streets["density"] = gdf_streets["contains_kgiop_objects"] / gdf_streets.length

select_columns = []
gdf_streets.nlargest(5, "density")

,geometry,contains_kgiop_objects,density
Название улицы,,,
Магдалинский переулок,"LINESTRING (3371179.834 8388009.593, 3371170.3...",5,0.066585
Волжский переулок,"MULTILINESTRING ((3371127.736 8386504.434, 337...",14,0.022229
Академический переулок,"MULTILINESTRING ((3371097.813 8385596.753, 337...",14,0.021333
Днепровский переулок,"LINESTRING (3371234.024 8385674.972, 3371231.3...",11,0.021283
7-я линия В.О.,"LINESTRING (3370595.996 8387058.387, 3370622.3...",19,0.019892


Переименуем столбцы *contains_kgiop_objects*	и *density* в русские названия

In [110]:
# DONE todo переименовать столбцы contains_kgiop_objects	и density в русские названия
rename_columns = {
    "contains_kgiop_objects": "Кол-во объектов культурного наследия",
    "density": "Плотность объектов культурного наследия на улице",
}
gdf_streets.rename(columns=rename_columns, inplace=True)
gdf_streets

,geometry,Кол-во объектов культурного наследия,Плотность объектов культурного наследия на улице
Название улицы,,,
1-я линия В.О.,"MULTILINESTRING ((3371228.480 8388213.098, 337...",42,0.016447
10-11-я линии В.О.,"MULTILINESTRING ((3369235.249 8388117.669, 336...",24,0.007373
12-13-я линии В.О.,"MULTILINESTRING ((3370596.664 8385143.832, 337...",22,0.006398
14-15-я линии В.О.,"LINESTRING (3369473.640 8386432.496, 3369476.8...",21,0.012094
14-я линия В.О.,"LINESTRING (3369473.640 8386432.496, 3369470.2...",5,0.002621
...,...,...,...
улица Нахимова,"MULTILINESTRING ((3364793.702 8386752.080, 336...",0,0.000000
улица Одоевского,"MULTILINESTRING ((3368096.740 8388961.412, 336...",0,0.000000
улица Репина,"MULTILINESTRING ((3371298.233 8387498.912, 337...",31,0.019799


Визуализируем на карте распределение объектов культурного наследия по улицам района

In [112]:
m = gdf_streets.explore("Кол-во объектов культурного наследия", tiles=TILES)
m = territory.explore(m=m, style_kwds={"fill": False, "weight": 5})

m

Сохраним датафрейм gdf_streets в файл streets_with_contains_kgiop.geojson в формате geojson

In [113]:
# todo сохранить датафрейм gdf_streets в файл streets_with_contains_kgiop.geojson в формате geojson
gdf_streets.to_file('streets_with_contains_kgiop.geojson', driver='GeoJSON')
